# Explore here


In [1]:
import pandas as pd
file = "/workspace/Time-Series-Acea-Smart-Water-Analytics-/data/raw/Aquifer_Auser.csv"
total_data = pd.read_csv(file)

1. Analisis descrptivo de datos

In [2]:
total_data.head()

,Date,Rainfall_Gallicano,Rainfall_Pontetetto,Rainfall_Monte_Serra,Rainfall_Orentano,Rainfall_Borgo_a_Mozzano,Rainfall_Piaggione,Rainfall_Calavorno,Rainfall_Croce_Arcana,Rainfall_Tereglio_Coreglia_Antelminelli,...,Temperature_Monte_Serra,Temperature_Ponte_a_Moriano,Temperature_Lucca_Orto_Botanico,Volume_POL,Volume_CC1,Volume_CC2,Volume_CSA,Volume_CSAL,Hydrometry_Monte_S_Quirico,Hydrometry_Piaggione
0,05/03/1998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,06/03/1998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,10.05,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,07/03/1998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9.2,0.0,10.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,08/03/1998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,11.4,0.0,13.85,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,09/03/1998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,11.4,0.0,12.85,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
total_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8154 entries, 0 to 8153
Data columns (total 27 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Date                                     8154 non-null   object 
 1   Rainfall_Gallicano                       5295 non-null   float64
 2   Rainfall_Pontetetto                      5295 non-null   float64
 3   Rainfall_Monte_Serra                     5289 non-null   float64
 4   Rainfall_Orentano                        5295 non-null   float64
 5   Rainfall_Borgo_a_Mozzano                 5295 non-null   float64
 6   Rainfall_Piaggione                       4930 non-null   float64
 7   Rainfall_Calavorno                       5295 non-null   float64
 8   Rainfall_Croce_Arcana                    5295 non-null   float64
 9   Rainfall_Tereglio_Coreglia_Antelminelli  5295 non-null   float64
 10  Rainfall_Fabbriche_di_Vallico            5295 no

In [8]:
# Análisis descriptivo de datos
data_info = total_data.info()
missing_values = total_data.isnull().sum()
descriptive_stats = total_data.describe()

data_info, missing_values, descriptive_stats

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8154 entries, 0 to 8153
Data columns (total 27 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Date                                     8154 non-null   object 
 1   Rainfall_Gallicano                       5295 non-null   float64
 2   Rainfall_Pontetetto                      5295 non-null   float64
 3   Rainfall_Monte_Serra                     5289 non-null   float64
 4   Rainfall_Orentano                        5295 non-null   float64
 5   Rainfall_Borgo_a_Mozzano                 5295 non-null   float64
 6   Rainfall_Piaggione                       4930 non-null   float64
 7   Rainfall_Calavorno                       5295 non-null   float64
 8   Rainfall_Croce_Arcana                    5295 non-null   float64
 9   Rainfall_Tereglio_Coreglia_Antelminelli  5295 non-null   float64
 10  Rainfall_Fabbriche_di_Vallico            5295 no

(None,
 Date                                          0
 Rainfall_Gallicano                         2859
 Rainfall_Pontetetto                        2859
 Rainfall_Monte_Serra                       2865
 Rainfall_Orentano                          2859
 Rainfall_Borgo_a_Mozzano                   2859
 Rainfall_Piaggione                         3224
 Rainfall_Calavorno                         2859
 Rainfall_Croce_Arcana                      2859
 Rainfall_Tereglio_Coreglia_Antelminelli    2859
 Rainfall_Fabbriche_di_Vallico              2859
 Depth_to_Groundwater_LT2                   3352
 Depth_to_Groundwater_SAL                   3609
 Depth_to_Groundwater_PAG                   4347
 Depth_to_Groundwater_CoS                   3839
 Depth_to_Groundwater_DIEC                  4884
 Temperature_Orentano                          0
 Temperature_Monte_Serra                       0
 Temperature_Ponte_a_Moriano                   0
 Temperature_Lucca_Orto_Botanico               0
 Volume_POL  

Feature engineering

In [13]:
import pandas as pd

# Convertir la columna 'Date' a formato de fecha
total_data['Date'] = pd.to_datetime(total_data['Date'], dayfirst=True)

# Verificar la conversión
data_types_after_conversion = total_data.dtypes

# Tratamiento de valores faltantes: para este análisis inicial, vamos a reemplazar los valores faltantes
# con la media de cada columna. Esta es una estrategia simple y común, aunque no siempre es la óptima.
data_filled = total_data.fillna(total_data.mean())

# Verificar el resultado del tratamiento de valores faltantes
missing_values_after_filling = data_filled.isnull().sum()

# Mostrar solo las primeras 10 columnas por brevedad
data_types_after_conversion.head(10), missing_values_after_filling.head(10)


(Date                                       datetime64[ns]
 Rainfall_Gallicano                                float64
 Rainfall_Pontetetto                               float64
 Rainfall_Monte_Serra                              float64
 Rainfall_Orentano                                 float64
 Rainfall_Borgo_a_Mozzano                          float64
 Rainfall_Piaggione                                float64
 Rainfall_Calavorno                                float64
 Rainfall_Croce_Arcana                             float64
 Rainfall_Tereglio_Coreglia_Antelminelli           float64
 dtype: object,
 Date                                       0
 Rainfall_Gallicano                         0
 Rainfall_Pontetetto                        0
 Rainfall_Monte_Serra                       0
 Rainfall_Orentano                          0
 Rainfall_Borgo_a_Mozzano                   0
 Rainfall_Piaggione                         0
 Rainfall_Calavorno                         0
 Rainfall_Croce_Arcana    

Eda